# Libraries & Settings

In [1]:
%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
import qgrid
#import beakerx as bx
from datetime import datetime as dtt

from pa_lib.file import data_files, load_bin, store_bin, store_excel
from pa_lib.data import (calc_col_partitioned, clean_up_categoricals, flatten, 
                         replace_col, cond_col, desc_col, unfactorize)
from pa_lib.util import obj_size, cap_words
from pa_lib.log  import time_log, info
from pa_lib.types import dtFactor
from pa_lib.vis import dive

# display long columns completely
pd.set_option('display.max_colwidth', 200)

pd.set_option('display.max_rows', 200)

# Load data

In [2]:
bd_raw = load_bin('bd_data.feather').rename(mapper=lambda name: cap_words(name, sep='_'), axis='columns')
bd = bd_raw.loc[(bd_raw.Netto >= 0)].pipe(clean_up_categoricals)

2019-08-07 10:32:07 [INFO] Reading from file C:\Users\kpf\data\bd_data.feather
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
2019-08-07 10:32:08 [INFO] Finished loading binary file in 0.72s (0.42s CPU)


In [3]:
desc_col(bd)

,DTYPE,NULLS,UNIQUE
Endkunde_NR,category,0/1526093,41740
Endkunde,category,0/1526093,39122
EK_Abc,category,5389/1520704,8
EK_Boni,category,2408/1523685,3
EK_Plz,category,475/1525618,3468
EK_Ort,category,474/1525619,3504
EK_Land,category,474/1525619,60
EK_HB_Apg_Kurzz,category,67234/1458859,95
EK_Aktiv,category,0/1526093,2
Agentur,category,770186/755907,4606


# Prepare Endkunden Information

In [45]:
def last_notna(s):
    try:
        return s.loc[s.notna()].iat[-1]
    except IndexError:
        return np.NaN

    
def collect(s, sep=', '):
    return sep.join(map(str, s[s.notna()].unique()))


# this takes around 150 seconds
with time_log('preparing EK_INFO'):
    ek_info = (bd.sort_values(['Endkunde_NR', 'Kampagne_Erfassungsdatum'])
               .astype({'Endkunde_NR': 'int64', 'Kamp_Erfass_Jahr': 'int16'})
               .groupby('Endkunde_NR')
               .agg({'Endkunde': last_notna,
                     'EK_Aktiv': last_notna,
                     'EK_Land': last_notna,
                     'EK_Plz': last_notna,
                     'EK_Ort': last_notna,
                     'Agentur': last_notna,
                     'Endkunde_Branchengruppe': last_notna,
                     'Endkunde_Branchengruppe_ID': last_notna,
                     'Auftrag_Branchengruppe_ID': [collect, 'nunique'],
                     'Kamp_Erfass_Jahr': ['min', 'max']
                     }))

ek_info.set_axis(labels='Endkunde EK_Aktiv EK_Land EK_Plz EK_Ort Agentur EK_BG EK_BG_ID Auftrag_BG_ID Auftrag_BG_Anz Kamp_Erfass_Jahr_min Kamp_Erfass_Jahr_max'.split(), 
                 axis='columns', inplace=True)

2019-08-07 11:27:27 [INFO] Finished preparing EK_INFO in 147.91s (150.56s CPU)


In [46]:
ek_info.head(10)

,Endkunde,EK_Aktiv,EK_Land,EK_Plz,EK_Ort,Agentur,EK_BG,EK_BG_ID,Auftrag_BG_ID,Auftrag_BG_Anz,Kamp_Erfass_Jahr_min,Kamp_Erfass_Jahr_max
Endkunde_NR,,,,,,,,,,,,
100034,Maurer + Salzmann AG,1,SCHWEIZ,8408,Winterthur,Maurer + Salzmann AG,WG - Dienstleistung,720,"720, 901, 405, 990, 400, 295, 410, 447, 509, 469, 436, 475, 460, 503, 504, 556, 610, 780",18,2009,2019
100039,Bank BSU Genossenschaft,1,SCHWEIZ,8610,Uster,"walder, werber werbeagentur ag",WG - Finanzwirtschaft,470,"470, 720",2,2009,2019
100043,Werbeagentur Knaus & Knaus BSW AG,0,SCHWEIZ,8604,Volketswil,NaN,WG - Dienstleistung,720,"901, 295",2,2011,2011
100045,Stoz Werbeagentur AG,1,SCHWEIZ,8330,Pfäffikon ZH,NaN,WG - Dienstleistung,720,"980, 469, 990",3,2011,2015
100061,Mundwiler Juwelen AG,1,SCHWEIZ,8400,Winterthur,NaN,WG - Persönlicher Bedarf,555,555,1,2009,2009
100064,Senn Communication,1,SCHWEIZ,8610,Uster,NaN,WG - Dienstleistung,720,"901, 990, 960",3,2009,2017
100066,"walder, werber werbeagentur ag",1,SCHWEIZ,8610,Uster,"walder, werber werbeagentur ag",WG - Dienstleistung,720,"400, 720, 990, 780, 366, 436, 295, 980, 901, 470, 332, 150, 795, 405",14,2009,2019
100068,Naef Marketing Kommunikation AG,1,SCHWEIZ,8400,Winterthur,NaN,WG - Dienstleistung,720,"720, 901",2,2009,2015
100077,Romer AG,1,SCHWEIZ,8400,Winterthur,NaN,WG - Persönlicher Bedarf,555,555,1,2012,2012


In [50]:
pd.crosstab(index=ek_info.Kamp_Erfass_Jahr_min, columns=ek_info.Kamp_Erfass_Jahr_max, margins=True)

Kamp_Erfass_Jahr_max,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,All
Kamp_Erfass_Jahr_min,,,,,,,,,,,,,
2007,0,4,0,0,1,0,1,3,0,0,2,4,15
2008,365,201,139,106,101,134,133,127,159,163,405,794,2827
2009,0,2210,662,433,345,385,324,368,338,406,812,1355,7638
2010,0,0,1693,406,233,183,189,164,171,175,358,418,3990
2011,0,0,0,1822,325,214,175,224,178,156,309,407,3810
2012,0,0,0,0,1638,326,217,191,224,159,266,311,3332
2013,0,0,0,0,0,1663,376,215,196,200,292,315,3257
2014,0,0,0,0,0,0,1685,374,229,198,353,343,3182
2015,0,0,0,0,0,0,0,1799,362,256,366,365,3148


### Distribution of Auftragsart vs. Vertrag

In [77]:
display(bd.pivot_table(index=['Auftragsart', 'Vertrag'], columns='Kamp_Erfass_Jahr',aggfunc='size', fill_value=0))
display('Netto-Umsatz')
display(bd.pivot_table(index=['Auftragsart', 'Vertrag'], columns='Kamp_Erfass_Jahr', values='Netto', aggfunc='sum', fill_value=0))

,Kamp_Erfass_Jahr,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Auftragsart,Vertrag,,,,,,,,,,,,,
Aushangauftrag Partner,Nein,0,90,2074,1698,1815,2337,2786,1520,33,16,19,23,14
Eigenwerbung APG,Nein,0,0,294,69,124,177,41,721,1256,1380,1168,423,138
Freespace,Nein,0,0,0,0,0,0,0,0,0,0,2786,4069,1704
Goodwill,Nein,0,10,103,104,123,180,163,134,192,134,150,158,61
Karitativ,Nein,0,200,438,695,403,704,524,726,897,1301,1354,1541,746
Kommerziell,Nein,0,14150,71746,78234,84716,78514,77696,91767,108510,126063,125936,126373,74202
Logistik für Dritte,Nein,0,0,96,525,1434,1416,1292,1530,1900,1866,2045,1772,810
Politisch,Nein,0,1041,3880,4385,8983,5950,5655,6170,10853,9185,6759,7973,6969
Promotion,Nein,0,191,3895,2870,3123,3301,5370,2590,3584,4673,5462,7906,988


'Netto-Umsatz'

,Kamp_Erfass_Jahr,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Auftragsart,Vertrag,,,,,,,,,,,,,
Aushangauftrag Partner,Nein,0,30444,849600,791206,739514,1026394,1000669,439344,110412,92214,117954,150966,72198
Eigenwerbung APG,Nein,0,0,3897655,0,4670,1048532,0,360,133390,126540,1222454,72427,0
Freespace,Nein,0,0,0,0,0,0,0,0,0,0,0,0,0
Goodwill,Nein,0,0,6543,7939,4953,18668,2994,4762,4191,1732,17772,16741,5925
Karitativ,Nein,0,724644,1960077,1628917,1866012,2668615,2253943,2318474,2578634,3459117,2775404,3962032,2632140
Kommerziell,Nein,0,33806952,148051046,162393013,168628594,175031787,158519973,182560236,193519765,204222602,197001081,191949492,145654263
Logistik für Dritte,Nein,0,0,0,0,4600,184578,31589,132394,479850,420927,418245,522661,239121
Politisch,Nein,0,2040430,6938949,6463858,18562173,11082670,9945225,10458567,16977360,11414298,6379246,8489070,10458705
Promotion,Nein,0,1083041,5375644,4376195,3393104,3505266,5451654,3185928,3787542,4991362,4881049,9304271,1960374
